In [ ]:
import os
import openai
from llama_index import ServiceContext
from llama_index.llms import OpenAI


# import
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index.embeddings import HuggingFaceEmbedding
from IPython.display import Markdown, display
import os
import urllib.request
import chromadb
from dotenv import load_dotenv
load_dotenv()


from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    ServiceContext,
    load_index_from_storage,
)
from llama_index.llms import OpenAI

from llama_index.tools import QueryEngineTool, ToolMetadata


openai.api_key = os.getenv("OPENAI_API_KEY")
llm = OpenAI(temperature=0, model="gpt-3.5-turbo-0613")
# llm = OpenAI(temperature=0, model="gpt-4-0613")
service_context = ServiceContext.from_defaults(llm=llm)

gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo-0613", temperature=0.3)
)
gpt4_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-4-0613", temperature=0.3)
)


from llama_index.callbacks import CallbackManager

callback_manager = CallbackManager([])

gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo-0613", temperature=0.3),
    callback_manager=callback_manager,
)
gpt_4_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-4-0613", temperature=0.3),
    callback_manager=callback_manager,
)


from llama_index.node_parser import SentenceSplitter
from llama_index import VectorStoreIndex

In [ ]:
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import SimpleDirectoryReader, ServiceContext
from llama_index import VectorStoreIndex, SummaryIndex, SimpleKeywordTableIndex
from llama_index.composability import ComposableGraph
from llama_index.llms import OpenAI

reader = SimpleDirectoryReader("./necrovox_docs/")
documents = reader.load_data()


from llama_index.embeddings import OpenAIEmbedding
from llama_index import ServiceContext, set_global_service_context

embed_model = OpenAIEmbedding(embed_batch_size=10)

service_context = ServiceContext.from_defaults(embed_model=embed_model)

# optionally set a global service context
set_global_service_context(service_context)

import chromadb.utils.embedding_functions as embedding_functions
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=OPENAI_API_KEY,
                model_name="text-embedding-ada-002"
            )


In [ ]:
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import SimpleDirectoryReader, ServiceContext
from llama_index import VectorStoreIndex, SummaryIndex, SimpleKeywordTableIndex
from llama_index.composability import ComposableGraph
from llama_index.llms import OpenAI

reader = SimpleDirectoryReader("./necrovox_docs/")
documents = reader.load_data()

In [ ]:
from llama_index.node_parser import SentenceSplitter

nodes = SentenceSplitter().get_nodes_from_documents(documents)

In [ ]:
chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection(name="necromunda_openai", embedding_function=openai_ef)
vector_store = ChromaVectorStore(chroma_collection=collection)
embed_model = OpenAIEmbedding(embed_batch_size=10)
service_context = ServiceContext.from_defaults(embed_model=embed_model)
# load from disk
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    service_context=service_context,
)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
from llama_index.tools.query_engine import QueryEngineTool

In [ ]:
query_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="necromunda_query_engine",
    description=(
        "Provides information about the Necromunda rules"
    )
)
query_engine_tools = [query_tool]


# ReAct Agent

In [ ]:
from llama_index.agent import ReActAgent
from llama_index.llms import OpenAI

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo")
# llm = OpenAI(model="gpt-4-0613")
base_agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

In [ ]:
response = base_agent.chat(
    "Cover the creation of a gang, from the basics of choosing a name, selecting a faction, creating gang fighters and selecting equipment for a valid 1000 point gang."
)
print(str(response))

In [ ]:
node_parser = SentenceSplitter()
nodes = node_parser.get_nodes_from_documents(documents)

In [ ]:
from llama_index.text_splitter import SentenceSplitter
from llama_index.extractors import TitleExtractor

node_parser = SentenceSplitter(chunk_size=512)
extractor = TitleExtractor()

# use transforms directly
nodes = node_parser(documents)

In [ ]:
from llama_index import ServiceContext, VectorStoreIndex
from llama_index.extractors import (
    TitleExtractor,
    QuestionsAnsweredExtractor,
)
from llama_index.ingestion import IngestionPipeline
from llama_index.text_splitter import TokenTextSplitter

transformations = [
    TokenTextSplitter(chunk_size=512, chunk_overlap=128),
    TitleExtractor(nodes=5),
    QuestionsAnsweredExtractor(questions=3),
]
text_splitter =  TokenTextSplitter(chunk_size=512, chunk_overlap=128)
title_extractor = TitleExtractor(nodes=5)
qa_extractor = QuestionsAnsweredExtractor(questions=3)

service_context = ServiceContext.from_defaults(
    transformations=[text_splitter, title_extractor, qa_extractor]
)

index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)

In [ ]:
index.storage_context.persist("./necrovox_index")

In [ ]:
from llama_index import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./necrovox_index")

# load index
index = load_index_from_storage(storage_context)
index.storage_context.persist(persist_dir="./necrovox_index")

In [ ]:
from llama_index.evaluation import (
    DatasetGenerator,
    QueryResponseDataset,
)

In [ ]:
from llama_index.finetuning.cross_encoders.cross_encoder import (
    CrossEncoderFinetuneEngine,
)

from llama_index.finetuning.cross_encoders.dataset_gen import (
    generate_ce_fine_tuning_dataset,
    generate_synthetic_queries_over_documents,
)


In [ ]:
documents[0].dict()

In [ ]:
# get 'queries' from train_dataset.json
import json
with open("./train_dataset.json", "r") as f:
    train_dataset = json.load(f)
    # queries = [q["query"] for q in train_dataset]

In [ ]:
train_dataset

In [ ]:
# Add the values of train_dataset['queries'] to a list
queries = []
for q in train_dataset['queries'].values():
    queries.append(q)

In [ ]:
from llama_index import VectorStoreIndex
from llama_index.prompts import PromptTemplate

qa_prompt_tmpl_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query.\n"
    "Query: {query_str}\n"
    "Answer: "
)
qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)


In [ ]:
queries[:500]

In [ ]:
from tqdm.notebook import tqdm


def augment_data_with_retrieval(dataset, retriever, separate_context=False):
    data_list = dataset.qr_pairs
    new_data_list = []
    for query_str, response in tqdm(data_list):
        retrieved_nodes = retriever.retrieve(query_str)
        retrieved_txts = [n.get_content() for n in retrieved_nodes]
        if separate_context:
            for retrieved_txt in retrieved_txts:
                fmt_query_str = qa_prompt_tmpl.format(
                    query_str=query_str, context_str=retrieved_txt
                )
                new_data_list.append((fmt_query_str, response))
        else:
            context_str = "\n\n".join(retrieved_txts)
            fmt_query_str = qa_prompt_tmpl.format(
                query_str=query_str, context_str=context_str
            )
            new_data_list.append((fmt_query_str, response))
    return new_data_list

In [ ]:
from llama_index.llama_dataset.generator import RagDatasetGenerator

In [ ]:
dataset_generator = RagDatasetGenerator.from_documents(
    documents=documents,
    service_context=service_context,
    num_questions_per_chunk=10,  # set the number of questions per nodes
)

In [ ]:
eval_dataset = dataset_generator.generate_dataset_from_nodes(num=10)

In [ ]:
eval_dataset.save_json("./eval_dataset.json")

In [ ]:
from llama_index.llama_dataset import (
    LabelledRagDataset,
    CreatedBy,
    CreatedByType,
    LabelledRagDataExample,
)


In [61]:
nest_asyncio.apply()
rag_dataset_generator = RagDatasetGenerator.from_documents(
    documents=documents,
    service_context=service_context,
    num_questions_per_chunk=1,  # set the number of questions per nodes
)

rag_dataset = rag_dataset_generator.generate_dataset_from_nodes()

  0%|          | 0/5 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 

 20%|██        | 1/5 [00:00<00:03,  1.28it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


 40%|████      | 2/5 [00:00<00:01,  2.58it/s]

...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


100%|█████████▉| 2119/2120 [31:01<00:00,  1.26it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


100%|██████████| 2120/2120 [31:05<00:00,  1.14it/s]


ImportError: Spacy is not installed, please install it with `pip install spacy`.

In [ ]:
from llama_index.llama_pack import download_llama_pack

RagEvaluatorPack = download_llama_pack("RagEvaluatorPack", "./pack")

In [ ]:
rag_evaluator = RagEvaluatorPack(
    query_engine=query_engine,  # built with the same source Documents as the rag_dataset
    rag_dataset=eval_dataset,
)

In [ ]:
eval_dataset = QueryResponseDataset.from_json("./eval_dataset.json")

In [ ]:
ce_datadet = generate_ce_fine_tuning_dataset(documents=documents, questions_list=queries[:500], max_chunk_length=1000, llm=llm, qa_doc_relevance_prompt=qa_prompt_tmpl, top_k=5,)

In [ ]:

final_finetuning_data_list = []
for doc in documents:
    questions_list = doc["questions"]
    documents = [Document(text=doc['text'])]
    local_finetuning_dataset = generate_ce_fine_tuning_dataset(
        documents=documents,
        questions_list=questions_list,
        max_chunk_length=256,
        top_k=5,
    )
    final_finetuning_data_list.extend(local_finetuning_dataset)

## Context with GPT-4

In [ ]:

eval_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-4", temperature=0), callback_manager=callback_manager
)

## Context With GPT-3.5

In [ ]:

eval_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0), callback_manager=callback_manager
)

In [ ]:
from llama_index.evaluation.rag import RagDatasetGenerator


In [ ]:
dataset_generator = DatasetGenerator(
    nodes[:39],
    service_context=eval_context,
    show_progress=True,
    num_questions_per_chunk=20,
)